### Neural Networks as math expressions:

- Neural Networks are just mathematical expressions. 
- They take as input (data & parameters) and output predictions which enter some loss function to measure (compare) them against true values.

A very simple example of a neural network might be a single neuron performing a linear regression task, which could be represented mathematically as:

$ y = wx + b $

We could add a non-linear activation function, such as the sigmoid function. The mathematical expression would then be:

$ y = \sigma(wx + b)$

where $\sigma$ is the sigmoid activation function defined as:

$ \sigma(z) = \frac{1}{1 + e^{-z}} $

- We want to tweak the parameters (knobs) to reduce the loss (and thus make NNs prediction close to truth).
- We know by how much to tweak parameters and in what direction via gradients (derivatives).  
- Backpropagation is an algorithm that computes the gradient / derivative of a loss function w.r.t parameters of the NN-math-expression.

### From Derivatives to Computation Graphs:

#### Derivatives

Once we know how to compute derivatives for simple functions, the next step is to compute derivatives for functions combinations. 

There are 3 ways to combine functions:

1. Sum: `h(x) = f(x) + g(x)`
2. Product: `h(x) = f(x) * g(x)` 
3. Function composition (one inside the other) `h(x) = f(g(x))`

<details>
    <summary> Their derivatives are given by (click to expand / collpase): </summary>

1. Sum rule: derivative of a sum = sum of derivatives >> easy to imagine: since we sum the "values" of both functions at each point, if we nudge each to produce some change, we'll end up summing the changes.
2. Product rule: dfirst x second + dsecond x first >> think of each function's value as representing a side of a square / rectangle.

```
       x^2
   +--------+
   |        |
sin(x)      sin(x)
   |        |
   +--------+
       x^2
```
and as we nudge x by dx such that -> x + dx

```
+---------- x^2 ---------dx^2---+
|                      |        |
|                      |        | sin(x)
|----------------------| -------| 
|                      |        | dsin(x)
+----------------------+--------+
            x^2          dx^2
```
ie: the resulting change will be the area of the 2 new (top right and bottom left) rectangles:
- x^2 * dsin(x)
- sin(x) * dx^2

(we ignore the small lower square dsin(x) * dx^2 since dx is supposed to be very small so in the limit ->0 this small square vanishes
</details>

3. Function composition:

From [Chain rule - Wikipedia](https://en.wikipedia.org/wiki/Chain_rule)
As put by George F. Simmons:

> "If a car travels twice as fast as a bicycle and the bicycle is four times as fast as a walking man, then the car travels 2 × 4 = 8 times as fast as the man."[^1]

Let $z, y, x$ be the (variable) positions of the car, the bicycle, and the walking man, respectively. The rate of change of relative positions of the car and the bicycle is

$ \frac{dz}{dy} = 2. $

Similarly,

$ \frac{dy}{dx} = 4. $

So, the rate of change of the relative positions of the car and the walking man is

$ \frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx} = 2 \cdot 4 = 8. $

The rate of change of positions is the ratio of the speeds, and the speed is the derivative of the position with respect to time; that is,

$ \frac{dz}{dx} = \frac{\frac{dz}{dt}}{\frac{dx}{dt}}, $

or, equivalently,

$ \frac{dz}{dt} = \frac{dz}{dx} \cdot \frac{dx}{dt}, $

which is also an application of the chain rule.

Now, Suppose we have the following math expression:

```
e = a * b
d = e + c
L = d * f
```

We want to calculate the derivative of `L` w.r.t `a & b` but also all other components `c, d, e, f`

How can we compute `dL/da` and similarly `dL/db`? 

```
de/da = b (since a is scaled '*' by b, if we nudge a a little bit, the result is also scaled by b
similarly de/db = a

dd/de = de/de + dc/de = 1 + 0 = 1
similarly dd/dc = 1

dL/dd = f
dL/df = d
```
What the chain rule then tells us is:
`dL/da = de/da x dL/de `

And we can do the same for all intermediate values. 

#### Computational Graphs:

**How can we turn the pen and paper math into computation inside the machine?**

Let's think about functional programming: [Source](https://link.springer.com/chapter/10.1007/978-1-4842-8853-5_6)
- A function takes an input and then produces an output. 
- The input of a function can be the output of another function.
- If we view a function as one node in a graph, and its input and output as incoming and outgoing links to other functions, respectively, as the computation continues, these functions are **chained** together to form a directed acyclic graph (DAG).

So, our functions will look like:
```
e = a * b
d = e + c
L = d * f
```
```
a ---  
      \  
       (*)-- e --(+)-- d --(*)-- L
      /           |         |
b ---             |         |
                  |      f--
              c---
```

And we can see that conveniently the **chain** rule allows us to break the complex computations of derivatives into simpler ones that can just be multiplied together. 

**How?**

Our goal is to compute the derivative of `L` w.r.t all other input values ie: `dL/dd, dL/df, dL/de, dL/dc, dL/da, dL/db`

At `L`, we can find `dL/dd` and `dL/df`. We can then pass `dL/dd` to `d` node. 

Similarly at `d` we can find `dd/de` and `dd/dc`. We then use `dL/dd` (passed from above) to compute:
- `dL/de = dL/dd x dd/de`
- `dL/dc = dL/dd x dd/dc`

And pass these results to `e` then at `e` we can find `de/da` and `de/db` and similarly compute `dL/da, dL/db`

### Computational implementation:

To realize this computationally, we'll implement a `Value` object as a wrapper around numbers. Numbers will be the terms in our neural network math expression. Each `Value` will know where it came from (its parents) and what operation produced it. Knowing the operation allows each value to compute its local derivative eg:
```
e = a + b
```
`e` knows it came from `a, b` through `+` so it can compute `de/da, de/db`

In [106]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [26]:
## let's start with simple demo
class Value:
    def __init__(self, data, _parents=(), op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_parents)
        self.op = op
        self.label = label

    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), op='+')
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), op='*')
        return out

    def __repr__(self):
        return f"Value(data={self.data})"

In [27]:
a = Value(2.0)
b = Value(4.0)
c = a + b

print(f"c is {c}")
print(c._prev)

c is Value(data=6.0)
{Value(data=2.0), Value(data=4.0)}


Let's add the ability to **compute gradients**: 

A `Value` object could be created manually `a = Value(2.0)` in which case its gradient is 0 by defualt or as a result of an operation on other `Value` objects eg `c = a + b` in which case it can propagate its local gradient to its parents.

Obviosuly it's a waste to compute the gradient for each `Value` so we'll implement gradient computation as a method to be called when needed.

In [29]:
class Value:
    def __init__(self, data, _parents=(), op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_parents)
        self.op = op
        self.label = label
        self._backward = lambda: None

    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), op='+')
        
        def _backward():
            self.grad = 1.0 * out.grad # compute local grad = 1.0 then apply chain rule to propagate
            other.grad = 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), op='*')

        def _backward():
            self.grad = other.data * out.grad
            other.grad = self.data * out.grad
        out._backward = _backward()
            
        return out

    def __repr__(self):
        return f"Value(data={self.data})"

In [38]:
## let's test
a = Value(2.0)
b = Value(4.0)
c = a + b
a, b, c, a.grad, b.grad, c.grad

(Value(data=2.0), Value(data=4.0), Value(data=6.0), 0.0, 0.0, 0.0)

In [43]:
c.grad = 1 #initialize dc/dc = 1
c._backward()
a, b, c, a.grad, b.grad, c.grad

(Value(data=2.0), Value(data=4.0), Value(data=6.0), 1.0, 1.0, 1)

In [48]:
# Now let's try another example
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'; b.grad = 1
b._backward()
a.grad, b.grad

(1.0, 1)

This is wrong. `b = a + a = 2a` so `db/da (a.grad) should = 2`

We have a bug in our code. Speciafically where it says:
```
 def __add__(self, other):
        out = Value(self.data + other.data, (self, other), op='+')
        
        def _backward():
            self.grad = 1.0 * out.grad # compute local grad = 1.0 then apply chain rule to propagate
            other.grad = 1.0 * out.grad
```
This will execute as follows:

```
b = a + a
self = a
other = a
self.grad = 1.0 * out.grad = 1.0 * 1.0 = 1.0 -> a.grad = 1.0
and exactly the same happens in the second step:
other.grad = 1.0 * out.grad = 1.0 * 1.0 = 1.0 -> a.grad = 1.0
```

Whenever a node is used more than once (eg if it's involved in multiple computations), it should accumulate the gradients it receives from the various points (and not reset). 

In [49]:
## the correct code:
class Value:
    def __init__(self, data, _parents=(), op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_parents)
        self.op = op
        self.label = label
        self._backward = lambda: None

    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), op='+')
        
        def _backward():
            self.grad += 1.0 * out.grad # now += 
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), op='*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward()
            
        return out

    def __repr__(self):
        return f"Value(data={self.data})"

In [50]:
# Now let's retest
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'; b.grad = 1
b._backward()
a.grad, b.grad

(2.0, 1)

<details> 

<summary>A side not on gradient accumulation (could be skipped): </summary>


In Multivariate Calculus, gradient accumulation refers to the process of accumulating or summing up the gradients of a multivariate function at different points. The gradient of a multivariate function is a vector that points in the direction of the steepest ascent of the function at a given point.

Here's how gradient accumulation works in Multivariate Calculus:

1. Consider a multivariate function f(x₁, x₂, ..., xₙ) that maps a vector of input variables (x₁, x₂, ..., xₙ) to a scalar output.

2. The gradient of the function f at a point (a₁, a₂, ..., aₙ) is denoted as ∇f(a₁, a₂, ..., aₙ) and is given by:
   ∇f(a₁, a₂, ..., aₙ) = (∂f/∂x₁, ∂f/∂x₂, ..., ∂f/∂xₙ) evaluated at (a₁, a₂, ..., aₙ)
   
   Where ∂f/∂xᵢ represents the partial derivative of f with respect to xᵢ.

3. Gradient accumulation involves calculating the gradients of the function f at multiple points and summing them up.

4. Let's say we have a set of points {(a₁, a₂, ..., aₙ), (b₁, b₂, ..., bₙ), ..., (k₁, k₂, ..., kₙ)} at which we want to accumulate the gradients.

5. We calculate the gradient at each point:
   - ∇f(a₁, a₂, ..., aₙ) = (∂f/∂x₁, ∂f/∂x₂, ..., ∂f/∂xₙ) evaluated at (a₁, a₂, ..., aₙ)
   - ∇f(b₁, b₂, ..., bₙ) = (∂f/∂x₁, ∂f/∂x₂, ..., ∂f/∂xₙ) evaluated at (b₁, b₂, ..., bₙ)
   - ...
   - ∇f(k₁, k₂, ..., kₙ) = (∂f/∂x₁, ∂f/∂x₂, ..., ∂f/∂xₙ) evaluated at (k₁, k₂, ..., kₙ)

6. The accumulated gradient is then obtained by summing up the gradients at each point:
   Accumulated Gradient = ∇f(a₁, a₂, ..., aₙ) + ∇f(b₁, b₂, ..., bₙ) + ... + ∇f(k₁, k₂, ..., kₙ)

The accumulated gradient represents the combined effect of the gradients at multiple points. It provides information about the overall direction and magnitude of the steepest ascent of the function considering the contributions from different points.
</details>

#### Automating backpropagation:

Let's automate this process of recursive application of the chain rule (a.k.a backpropagation a.k.a repeated calls to `._backward`

To do so, we'll need to order the nodes such that we start at the last node and only call `.backward` on some node if we've computed the gradient on all its dependencies (the ones after it in the graph).


In our simple example :
```
e = a * b
d = e + c
L = d * f
```
```
a ---  
      \  
       (*)-- e --(+)-- d --(*)-- L
      /           |         |
b ---             |         |
                  |      f--
              c---
```
A typical use case is calling `.backward()` on `L` and then the algorithm should figure out it need initialize the gradient of `L` to 1 (which represents `dL/dL`) then calls `L._backward()` then `d and f` and so on all the way to `a and b`

Sorting the nodes this way (left to right) as laid out in the graph so they have the correct order can be done using Topological Sort algorithm.

In [51]:
topo_sorted = []
visited = set()

def build_topo(node):
    if node not in visited:
        visited.add(node)
        for parent in node._prev:
            build_topo(parent)
        topo_sorted.append(node) # add a node if all its parent nodes are added

In [58]:
a = Value(1.0)
b = Value(2.0)
c = Value(3.0)
f = Value(4.0)

e = a * b
d = e + c
L = d * f

topo_sorted = []
visited = set()
build_topo(L)
topo_sorted

[Value(data=4.0),
 Value(data=3.0),
 Value(data=1.0),
 Value(data=2.0),
 Value(data=2.0),
 Value(data=5.0),
 Value(data=20.0)]

In [104]:
# let's add this to our code
class Value:
    def __init__(self, data, _parents=(), op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_parents)
        self.op = op
        self.label = label
        self._backward = lambda: None

    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), op='+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), op='*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
            
        return out

    def __repr__(self):
        return f"Value(data={self.data})"

    
    def backward(self):
        topo = []
        visited = set()
        
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for parent in node._prev:
                    build_topo(parent)
                topo.append(node)

        self.grad = 1.0 #initialize last node (current)
        build_topo(self) # build the graph topologically sorted
        for node in reversed(topo):
            node._backward()
    

In [105]:
# test
a = Value(1.0)
b = Value(2.0)
c = Value(3.0)
f = Value(4.0)

e = a * b
d = e + c
L = d * f

L.backward()
nodes_list = [a,b,c,d,e,f,L]
for node in nodes_list:
    print(node.grad)

8.0
4.0
4.0
4.0
4.0
5.0
1.0


#### Adding more operations / functions:

Now, we can add more operations. In fact, we can add any operation / function we want as long as we know how to compute its local gradients. 

<details>

<summary> NB: Notice what happens with `+` and `*` (<i>click to expand/collpase</i>) </summary>

```
    def __add__(self, other):
        ...
        
        def _backward():
            self.grad += 1.0 * out.grad 
            other.grad += 1.0 * out.grad
        ...
```
`+` is just routing / distributing the gradient it receives (`out.grad`)

```
    def __mul__(self, other):
        ...

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        ...
```
`*` is scaling the gradient by some factor eg `self.data` or `other.data`
</details>

Other functions to implement:
- `pow`
- `tanh`
- `exp`
- ` sub` and `div`

In [107]:
class Value:
    def __init__(self, data, _parents=(), op='', label=''):
        self.data = data
        self._prev = set(_parents)
        self.op = op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data:{self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), op='+')

        def _backward():
            self.grad +=  1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), op='*')

        def _backward():
            self.grad = other.data * out.grad
            other.grad = self.data * out.grad

        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visisted = set()
        def topo_sort(node):
            if node not in visited:
                visited.add(node)
                for parent in node._prev:
                    topo_sort(parent)
                topo.append(node)

        self.grad = 1.0
        topo_sort(self)
        for node in reversed(topo):
            node._backward()

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self, ), op=f'**{other}')

        def _backward():
            self.grad += other.data * (self.data** (other.data -1)) * out.grad

        out._backward = _backward
        return out

    def __tanh__(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), op='tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad

        return out

    # subtraction is addition to negative other
    def __neg__(self):
        return -1 * self
    def __sub__(self, other):
        return self + (-other)
    # division is multiplication by other^-1
    def __truediv__(self, other):
        return self * other**-1

    # make the operations work both ways x + y == y + x
    def __radd__(self, other):
        return self + other
    def __rmul__(self, other):
        return self * other
    def __rsub__(self, other):
        return other + (-self)
    def __rtruediv__(self, other):
        return other * self**-1
        